In [1]:
import random

import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import mnist
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Input, UpSampling2D
from keras.models import Model


def print_images(dataset, nbr=1):
    for _ in range (nbr):
        item = random.randrange(0, dataset.shape[0], dataset.shape[0]/10)
        plt.imshow(dataset[item], interpolation='nearest')
        plt.show()


def normalize_x_data(data):
    return np.reshape(data.astype('float32') / 255, (len(data), 28, 28, 1))


def noise_data(data, noise_factor):
    return np.clip(data + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=data.shape), 0.0, 1.0)


def build_auto_encoder():
    input_img = Input(shape=(28, 28, 1))
    layer = Convolution2D(
        kernel_size=(4, 4), filters=16, strides=(1, 1), border_mode="same"
    )(input_img)
    layer = MaxPooling2D((2, 2), border_mode="same")(layer)
    layer = Convolution2D(
        kernel_size=(4, 4), filters=32, strides=(1, 1), border_mode="same"
    )(layer)
    layer = MaxPooling2D((2, 2), border_mode="same")(layer)
    layer = Convolution2D(
        kernel_size=(3, 3), filters=16, strides=(1, 1), border_mode="same"
    )(layer)
    layer = UpSampling2D(size=(2, 2))(layer)
    layer = Convolution2D(
        kernel_size=(4, 4), filters=16, strides=(1, 1), border_mode="same"
    )(layer)
    layer = UpSampling2D(size=(2, 2))(layer)
    layer = Convolution2D(
        filters=1, kernel_size=3, strides=1, activation="sigmoid", border_mode="same"
    )(layer)
    model = Model(input_img, layer)
    model.compile(optimizer="adam", loss="binary_crossentropy")
    model.summary()
    return model


def build_ocr_model():
    input_img = Input(shape=(28, 28, 1))
    layer = Convolution2D(
        kernel_size=(4, 4), filters=16, strides=1, border_mode="same"
    )(input_img)
    layer = Convolution2D(
        kernel_size=(4, 4), filters=16, strides=1, border_mode="same"
    )(layer)
    layer = Convolution2D(
        kernel_size=(4, 4), filters=32, strides=1, border_mode="same"
    )(layer)
    layer = Convolution2D(
        kernel_size=(4, 4), filters=16, strides=1, border_mode="same"
    )(layer)
    layer = Convolution2D(
        kernel_size=(4, 4), filters=1, strides=1, border_mode="same"
    )(layer)
    model = Model(input_img, layer)
    model.compile(optimizer="adam", loss="binary_crossentropy")
    model.summary()
    return model


def load_data():
    (_x_train, _), (_x_test, _) = mnist.load_data()
    _x_train = normalize_x_data(_x_train)
    _x_test = normalize_x_data(_x_test)
    _noised_x_train = noise_data(_x_train, 0.9)
    _noised_x_test = noise_data(_x_test, 0.9)
    return _x_train, _x_test, _noised_x_train, _noised_x_test

Using TensorFlow backend.


In [4]:
x_train, x_test, noised_x_train, noised_x_test = load_data()

In [3]:
noised_x_test = noise_data(noised_x_test, 0.9)

In [5]:
noise_remover = build_auto_encoder()
ocr = build_auto_encoder()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 16)        272       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 16)        4112      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 32)        8224      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 16)        8208      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 1)         257       
Total params: 21,073
Trainable params: 21,07

/home/louis/anaconda3/envs/DeepLearningCourse/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(4, 4), filters=16, strides=1, padding="same")`
/home/louis/anaconda3/envs/DeepLearningCourse/lib/python3.6/site-packages/ipykernel_launcher.py:59: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(4, 4), filters=16, strides=1, padding="same")`
/home/louis/anaconda3/envs/DeepLearningCourse/lib/python3.6/site-packages/ipykernel_launcher.py:62: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(4, 4), filters=32, strides=1, padding="same")`
/home/louis/anaconda3/envs/DeepLearningCourse/lib/python3.6/site-packages/ipykernel_launcher.py:65: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(4, 4), filters=16, strides=1, padding="same")`
/home/louis/anaconda3/envs/DeepLearningCourse/lib/python3.6/site-packages/ipykernel_launcher

In [6]:
noise_remover.fit(
    x=noised_x_train,
    y=x_train,
    nb_epoch=3,
    batch_size=256,
    shuffle=True,
    validation_data=(noised_x_test, x_test)
)

/home/louis/anaconda3/envs/DeepLearningCourse/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys



Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 135s 2ms/step - loss: 0.2975 - val_loss: 0.2096

In [7]:
de_noised_x_train = noise_remover.predict(noised_x_train)
de_noised_x_test = noise_remover.predict(noised_x_test)

In [8]:
ocr.fit(
    x=de_noised_x_train,
    y=x_train,
    nb_epoch=3,
    batch_size=256,
    shuffle=True,
    validation_data=(de_noised_x_test, x_test)
)

/home/louis/anaconda3/envs/DeepLearningCourse/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 41s 677us/step - loss: 0.2317 - val_loss: 0.1851

In [9]:
ocrised_x_test = ocr.predict(de_noised_x_test)

In [10]:
auto_clarified_x_test = ocrised_x_test

In [11]:
for _ in range(20):
    auto_clarified_x_test = ocr.predict(auto_clarified_x_test)

In [1]:
row_nbr = 5
n = 10
plt.figure(figsize=(30,6))
for i in range(n):
    img_index = random.randint(0, len(x_test) - 1)

    ax = plt.subplot(row_nbr, n, i+1+n*0)
    plt.imshow(x_test[img_index].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title('original images')

    ax = plt.subplot(row_nbr, n, i+1+n*1)
    plt.imshow(noised_x_test[img_index].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title('noisy images')

    ax = plt.subplot(row_nbr, n, i+1+n*2)
    plt.imshow(de_noised_x_test[img_index].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title('denoised images')

    ax = plt.subplot(row_nbr, n, i+1+n*3)
    plt.imshow(ocrised_x_test[img_index].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title('clarified images')

    ax = plt.subplot(row_nbr, n, i+1+n*4)
    plt.imshow(auto_clarified_x_test[img_index].reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title('auto clarified images')

plt.show()

NameError: name 'plt' is not defined